In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [7]:
ratings_df = pd.read_csv('./small_data/ml-latest-small/ratings.csv')

In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
ratings_df.shape

(100836, 4)

In [10]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [11]:
new_df = ratings_df.drop(columns='timestamp')

In [12]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
data = Dataset.load_from_df(new_df,reader)

In [13]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


In [14]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise.prediction_algorithms import SVD, SVDpp
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

### Collaborative Filtering with Baseline Only

In [15]:
bo = BaselineOnly()

In [16]:
bo.fit(data)

Estimating biases using als...


AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

### Collaborative Filtering with Slope One

In [17]:
so = SlopeOne()

In [18]:
so.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_items'

### Collaborative Filtering Using SVD

In [19]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [20]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8685056261014086, 'mae': 0.6672967723947503}
{'rmse': {'n_factors': 20, 'reg_all': 0.02}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


### Collaborative Filtering with SVD++

In [ ]:
## Perform a gridsearch with SVDpp
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svdpp = GridSearchCV(SVDpp,param_grid=params,n_jobs=-1)
g_s_svdpp.fit(data)

In [ ]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svdpp.best_score)
print(g_s_svdpp.best_params)

### Collaborative Filtering Using Knn_basic

In [ ]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [ ]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

In [ ]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

In [ ]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

### Reccs with SVD

In [ ]:
df_movies = pd.read_csv('./data/movies.csv')

In [ ]:
df_movies.head()

In [ ]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [ ]:
svd.predict(2, 4)

### uSer ratings with SVD

In [ ]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      

In [ ]:
user_rating = movie_rater(df_movies, 4, 'Comedy')

### new users with SVD

In [ ]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [ ]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [ ]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [ ]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [ ]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)